In [1]:
import copy
import numpy as np
import torch
from tqdm import tqdm
from sklearn.utils import check_random_state
import pandas as pd
from proplace.clfutils import HiddenPrints
import proplace
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


D:\Apps\anaconda\envs\proplace\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
[WARNING] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.
 [lazy_loader.py _load]


In [2]:
# load dataset utils
dataset = proplace.InnDataSet("give_me_some_credit")

In [15]:
# get a clf#
torch.manual_seed(0)
np.random.seed(5)
random_state = check_random_state(10)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
d = dataset
torch_model = proplace.train_clf(d.X1_train, d.y1_train, d.X1_test, d.y1_test, 20, epochs=3, data_name="give_me_some_credit", save_clf=False,
                        load_clf=False)
model = proplace.InnModel(dataset, torch_model, 20)

Using cuda device
Epoch 0: train loss: 0.7437196969985962
Epoch 0: train loss: 0.7191342711448669
Epoch 0: train loss: 0.69671231508255
Epoch 0: train loss: 0.6771828532218933
Epoch 0: train loss: 0.6518386602401733
Epoch 0: train loss: 0.6321981549263
Epoch 0: train loss: 0.6206355690956116
Epoch 0: train loss: 0.5932130217552185
Epoch 0: train loss: 0.5650101900100708
Epoch 0: train loss: 0.5645900964736938
Epoch 0: train loss: 0.5264307856559753
Epoch 0: train loss: 0.4796767234802246
Epoch 0: train loss: 0.5112673044204712
Epoch 0: train loss: 0.4141991138458252
Epoch 0: train loss: 0.3740864396095276
Epoch 0: train loss: 0.3177039623260498
Epoch 0: train loss: 0.41362762451171875
Epoch 0: train loss: 0.3273945152759552
Epoch 0: train loss: 0.21375000476837158
Epoch 0: train loss: 0.26735934615135193
Epoch 0: train loss: 0.21889716386795044
Epoch 0: train loss: 0.2784709334373474
Epoch 0: train loss: 0.31628987193107605
Epoch 0: train loss: 0.23128177225589752
Epoch 0: train loss: 

In [16]:
m2s = proplace.retrain_models(dataset, 20, 3) + proplace.retrain_models_leave_some_out(dataset, 20, 3)

100%|██████████| 10/10 [00:40<00:00,  4.03s/it]


## Prepare for exps

In [17]:
from exputils import *
clf = model
test_xs_vals, test_xs, test_xs_carla, utildataset, nodes = get_test_data(model, m2s, dataset)

## Run Baselines

In [21]:
rnnce_ces, treer, X_class1_clf_robust = run_rnnce(d, clf, nodes, utildataset, 0.045, test_xs_vals)

56662it [23:33, 40.08it/s]


In [22]:
# ours
#rnnce_ces, treer, X_class1_clf_robust = run_rnnce(d, clf, nodes, utildataset, 0.045, test_xs_vals)
proplace_ces = run_proplace(clf, nodes, utildataset, 0.045, treer, X_class1_clf_robust, test_xs_vals)
proplace_scores = eval_ces(clf, m2s, proplace_ces, d, utildataset, test_xs_vals, 0.045)

50it [01:02,  1.24s/it]


1.0 >>> validity
1.0 >>> robustness - M2 validity


50it [02:40,  3.21s/it]

1.0 0.00529898026954905 >>> robustness - delta validity, avg bound
0.05754727296614758 >>> l1 cost
0.92 >>> percentage of inliers dataset class 1 points
1.2366107807867035 >>> avg lof dataset class 1 points


In [23]:
wach_params = {
            "lr": 0.1,
            "lambda_": 0.001,
            "n_iter": 100,
            "t_max_min": 0.5,
            "norm": 1,
            "clamp": False,
            "loss_type": "MSE",
            "y_target": [1],
            "binary_cat_features": False,
        }
wach_ces = run_wachter(copy.deepcopy(clf), copy.deepcopy(test_xs_carla), wach_params)
wach_scores = eval_ces(clf, m2s, wach_ces, d, utildataset, test_xs_vals,  delta=0.045)

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactua

50it [02:37,  3.14s/it]

0.18 -0.14287575065669705 >>> robustness - delta validity, avg bound
0.1481171087879949 >>> l1 cost
0.02 >>> percentage of inliers dataset class 1 points
2.8041599852762804 >>> avg lof dataset class 1 points


In [29]:
rbr_ces = run_rbr(clf, d, random_state, test_xs_vals)
rbr_scores = eval_ces(clf, m2s, rbr_ces, d, utildataset, test_xs_vals, 0.045)

50it [05:39,  6.79s/it]


1.0 >>> validity
0.9030000000000001 >>> robustness - M2 validity


50it [02:38,  3.17s/it]

0.0 -0.27059100796124036 >>> robustness - delta validity, avg bound
0.04969111654362668 >>> l1 cost
0.8 >>> percentage of inliers dataset class 1 points
1.304884117171781 >>> avg lof dataset class 1 points


In [28]:
robx_ces = run_robx(clf, d, test_xs_vals, tao=0.6)
robx_scores = eval_ces(clf, m2s, robx_ces, d, utildataset, test_xs_vals, 0.045)

56662it [01:08, 830.22it/s]


(53853, 10)


50it [00:01, 45.07it/s]


1.0 >>> validity
1.0 >>> robustness - M2 validity


50it [02:48,  3.38s/it]

0.96 0.23101571633164536 >>> robustness - delta validity, avg bound
0.07328271069198757 >>> l1 cost
0.76 >>> percentage of inliers dataset class 1 points
1.3500294402521293 >>> avg lof dataset class 1 points


In [49]:
roar_params = {
        "lr": 0.1,
        "lambda_": 0.08,
        "delta_max": 0.2,
        "norm": 1,
        "t_max_min": 3,
        "loss_type": "MSE",
        "y_target": [1],
        "binary_cat_features": False,
        "loss_threshold": 1e-3,
        "discretize": False,
        "sample": False,
        "lime_seed": 0,
        "seed": 0,
    }
roar_ces = run_roar(clf, test_xs_carla, roar_params)
roar_scores = eval_ces(clf, m2s, roar_ces, d, utildataset, test_xs_vals, delta=0.045)

[INFO] Start generating LIME coefficients [model.py get_counterfactuals]
[INFO] Finished generating LIME coefficients [model.py get_counterfactuals]
0.96 >>> validity
0.9550000000000004 >>> robustness - M2 validity


50it [02:41,  3.24s/it]

0.875 1.0550976057825483 >>> robustness - delta validity, avg bound
0.18777945757417291 >>> l1 cost
0.041666666666666664 >>> percentage of inliers dataset class 1 points
4.220597524815384 >>> avg lof dataset class 1 points


In [37]:
protor_ces = run_proto_r(clf, d, utildataset, test_xs_vals, nodes, delta_target=0.045)
protor_scores = eval_ces(clf, m2s, protor_ces, d, utildataset, test_xs_vals, 0.045)

0it [00:00, ?it/s]

[WARNING] From D:\Apps\anaconda\envs\proplace\lib\site-packages\alibi\explainers\cfproto.py:123: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.
 [module_wrapper.py _tfmw_add_deprecation_warning]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]


1it [00:50, 50.84s/it]

[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]


2it [01:42, 51.21s/it]

...

[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]


50it [45:25, 54.50s/it]


1.0 >>> validity
1.0 >>> robustness - M2 validity


50it [02:38,  3.16s/it]

1.0 0.6436884312307138 >>> robustness - delta validity, avg bound
0.06581595642719576 >>> l1 cost
0.62 >>> percentage of inliers dataset class 1 points
1.4865195067101893 >>> avg lof dataset class 1 points


In [36]:
milpr_ces = run_milpr(clf, d, nodes, utildataset, 0.08, test_xs_vals)
milpr_scores = eval_ces(clf, m2s, milpr_ces, d, utildataset, test_xs_vals, 0.045)

50it [01:26,  1.72s/it]


1.0 >>> validity
1.0 >>> robustness - M2 validity


50it [02:33,  3.07s/it]

1.0 1.339208804254732 >>> robustness - delta validity, avg bound
0.058646908382038815 >>> l1 cost
0.18 >>> percentage of inliers dataset class 1 points
2.0815658040120395 >>> avg lof dataset class 1 points


In [38]:
# make table
from tabulate import tabulate
scores_names = ["name", "validity", "delta validity", "M2 validity", "l1", "%inlier class 1", "lof class 1"]
scores_table = [scores_names,
                np.concatenate((['wach'], np.round(wach_scores, 3))),
                np.concatenate((['rbr'], np.round(rbr_scores, 3))),
                np.concatenate((['robx'], np.round(robx_scores, 3))),
                np.concatenate((['roar'], np.round(roar_scores, 3))),
                np.concatenate((['proto-r'], np.round(protor_scores, 3))),
                np.concatenate((['milp-r'], np.round(milpr_scores, 3))),
                np.concatenate((['proplace'], np.round(proplace_scores, 3))), ]
print(tabulate(scores_table, headers='firstrow', tablefmt='outline'))

+----------+------------+------------------+---------------+-------+-------------------+---------------+
| name     |   validity |   delta validity |   M2 validity |    l1 |   %inlier class 1 |   lof class 1 |
+==========+============+==================+===============+=======+===================+===============+
| wach     |          1 |             0.18 |         0.839 | 0.148 |              0.02 |         2.804 |
| rbr      |          1 |             0    |         0.903 | 0.05  |              0.8  |         1.305 |
| robx     |          1 |             0.96 |         1     | 0.073 |              0.76 |         1.35  |
| roar     |          1 |             0.82 |         0.998 | 0.226 |              0    |        12.203 |
| proto-r  |          1 |             1    |         1     | 0.066 |              0.62 |         1.487 |
| milp-r   |          1 |             1    |         1     | 0.059 |              0.18 |         2.082 |
| proplace |          1 |             1    |         1 

In [ ]:
--------+---------------+-------+-------------------+--------------